In [1]:
# 1. import packages

import pandas as pd
import numpy as np
from pulp import *

In [2]:
# import data
students_df = pd.read_csv('data/example_subject_students.csv')
schools_df = pd.read_csv('data/example_subject_schools.csv')


In [ ]:
# do pre-processing
# from https://github.com/UCL/ioe-student-school-allocation

